In [1]:
import pandas as pd

In [2]:
fmpg_filename = 'input20190507.xlsx'
# 读入数据
fmct = pd.read_excel('FMCT.xlsx')
fmop = pd.read_excel('FMOP.xlsx')
fmvs = pd.read_excel('FMVS.xlsx')
frav = pd.read_excel('FRAV.xlsx')
fmpg = pd.read_excel(fmpg_filename, dtype=str).replace('nan', '')
fmpg = fmpg.applymap(lambda x: x.strip())

In [3]:
# 重命名参考表列名
nfmvs = pd.DataFrame(fmvs[['VSAPPL', 'VSNUMDOC', 'VSTITRE']])
nfmct = pd.DataFrame(fmct[['CTAPPL', 'CTNUMDOC', 'CTTITRE']])
nfmop = pd.DataFrame(fmop[['OPAPPL', 'OPNUMDOC', 'OPTITRE']])
nfmvs.rename(columns={'VSAPPL': 'PROTOCOL', 'VSNUMDOC': 'REFERENCE', 'VSTITRE': 'TITLE'}, inplace=True)
nfmct.rename(columns={'CTAPPL': 'PROTOCOL', 'CTNUMDOC': 'REFERENCE', 'CTTITRE': 'TITLE'}, inplace=True)
nfmop.rename(columns={'OPAPPL': 'PROTOCOL', 'OPNUMDOC': 'REFERENCE', 'OPTITRE': 'TITLE'}, inplace=True)
docs = {'1': nfmvs, '2': nfmct, '3': nfmop}

In [4]:
# 日期格式化
def performace_dates_join(years, months, days):
    ll = []
    for y, m, d in zip(years, months, days):
        ll.append(y + m + d)
    return pd.Series(ll)

def deadline_dates_join(years, months, days):
    ll = []
    for y, m, d in zip(years, months, days):
        if y != '' and m != '' and d != '':
            ll.append(str(int(y)) + '/' + str(int(m)) + '/' + str(int(d)))
        else:
            ll.append('')
    return pd.Series(ll)

In [5]:
frav = frav.applymap(lambda x: x.strip())
fmpg['AVOFFICIEL'] = fmpg.merge(frav[['AVNUMAP', 'AVOFFICIEL']], how='left', left_on='PGNUMAP', right_on='AVNUMAP')['AVOFFICIEL']

In [6]:
# 按导出列名重构fmpg
d = {'AC': fmpg['AVOFFICIEL'].str.strip(),
     'PROTOCOL': fmpg['PGAPPL'].str.strip(),
     'PGCODTE': fmpg['PGCODTE'],
     'REFERENCE': fmpg['PGNUMDOC'].str.strip(),
     'STATUS': fmpg['PGSIT'].str.strip(),
     'PERFORMACE_DOC': fmpg['PGAPPDOC1'].str.strip() + fmpg['PGAPPDOC2'].str.strip(),
     'PERFORMACE_DATE': performace_dates_join(fmpg['PGAPPA4'].str.strip(), fmpg['PGAPPMM'].str.strip(), fmpg['PGAPPJJ'].str.strip()),
     'FH_DEADLINE': fmpg['PGPRVH'],
     'CY_DEADLINE': fmpg['PGPRVC'],
     'DEADLINE_DOC': fmpg['PGPRVDOC1'].str.strip() + fmpg['PGPRVDOC2'].str.strip(),
     'DEADLINE_DATE': deadline_dates_join(fmpg['PGPRVA4'].str.strip(), fmpg['PGPRVMM'].str.strip(), fmpg['PGPRVJJ'].str.strip())
    }

nfmpg = pd.DataFrame(d, columns=['AC', 'PROTOCOL', 'PGCODTE', 'REFERENCE', 'STATUS', 'PERFORMACE_DOC', 'PERFORMACE_DATE', 
                              'FH_DEADLINE', 'CY_DEADLINE', 'DEADLINE_DOC', 'DEADLINE_DATE'])

In [7]:
# 根据doc类型分组
groupbycode = nfmpg.groupby(nfmpg['PGCODTE'])
dfs = {}
for k, v in groupbycode:
    dfs[k] = pd.DataFrame(v)

In [8]:
# 根据REFERENCE获取TITLE
mindex = ['AC', 'PROTOCOL', 'PGCODTE', 'REFERENCE', 'TITLE', 'STATUS', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE']

merged_dfs = {}
for k, v in dfs.items():
    merged_dfs[k] = pd.merge(v, docs[k], on=['PROTOCOL', 'REFERENCE'], how='left')[mindex]

In [9]:
not_filtered_df = merged_dfs['2']

In [55]:
ac = not_filtered_df['AC'].drop_duplicates().sort_values().reset_index(drop=True)
numbers = pd.Series([x for x in range(1, 136)])
nums = pd.concat([ac, numbers], ignore_index=True, axis=1)
nums.columns = ['AC', 'NUM']
numbered_df = pd.merge(not_filtered_df, nums)
numbered_df = numbered_df.sort_values('NUM')

In [63]:
numbered_df[(numbered_df['NUM'] > 5) & (numbered_df['NUM'] <= 10)]

,AC,PROTOCOL,PGCODTE,REFERENCE,TITLE,STATUS,PERFORMACE_DOC,DEADLINE_DOC,FH_DEADLINE,CY_DEADLINE,PERFORMACE_DATE,DEADLINE_DATE,NUM
8552,B-1661,A320,2,553003-01-1,VERT STABILIZER CENTER BOX,O,,,0,0,,2027/3/28,6
7842,B-1661,A320,2,200127-02-1,REAR AVIONIC COMPARTMENT,O,,,0,0,,2021/3/29,6
7841,B-1661,A320,2,200125-06-1,LATERAL AVIONIC COMPARTMENT,O,,,0,0,,2021/3/29,6
7840,B-1661,A320,2,200125-05-1,LATERAL AVIONIC COMPARTMENT,O,,,0,0,,2021/3/29,6
7839,B-1661,A320,2,200125-01-1,LATERAL AVIONIC COMPARTMENT,O,,,0,0,,2021/3/29,6
7838,B-1661,A320,2,200121-04-1,FORWARD AVIONIC COMPARTMENT,O,,,0,0,,2021/3/29,6
7837,B-1661,A320,2,200121-03-1,FORWARD AVIONIC COMPARTMENT,O,,,0,0,,2021/3/29,6
7836,B-1661,A320,2,200121-01-1,FORWARD AVIONIC COMPARTMENT,O,CS00960129,,0,0,20190311,2023/3/10,6
7835,B-1661,A320,2,200003-03-1,HIRF/LIGHTNING PROTECTION-VERTICAL FIN,O,,,0,0,,2021/3/29,6
7834,B-1661,A320,2,200003-02-1,HIRF/LIGHTNING PROTECTION-VERTICAL FIN,O,,,0,0,,2021/3/29,6


In [73]:
total = 135
format_filename = 'output_{}.xlsx'
for x, y in enumerate(range(5, 136, 5)):
    filename = output_name.format(str(x + 1).zfill(2))
    numbered_df[(numbered_df['NUM'] > y - 5) & (numbered_df['NUM'] <= y)].to_excel(filename, columns=['AC', 'PROTOCOL', 'REFERENCE', 'TITLE', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE'])

In [9]:
filtered_df = merged_dfs['2']
filtered_df = filtered_df[filtered_df['PERFORMACE_DATE'].str.isalnum()]
filtered_df.to_excel('filtered_output_20190507.xlsx', columns=['AC', 'PROTOCOL', 'REFERENCE', 'TITLE', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE'])

In [10]:
not_filtered_df = merged_dfs['2']
not_filtered_df.to_excel('not_filtered_output_20190507.xlsx', columns=['AC', 'PROTOCOL', 'REFERENCE', 'TITLE', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE'])

In [11]:
model_output = merged_dfs['2']
model_output = model_output[model_output['PERFORMACE_DATE'].str.isalnum()]
model_output.to_excel('model_output_20190507.xlsx', columns=['PROTOCOL', 'REFERENCE', 'AC', 'PERFORMACE_DATE'])

In [38]:
merged_dfs['2']

,AC,PROTOCOL,PGCODTE,REFERENCE,TITLE,STATUS,PERFORMACE_DOC,DEADLINE_DOC,FH_DEADLINE,CY_DEADLINE,PERFORMACE_DATE,DEADLINE_DATE
0,B-6325,A320,2,EOA320-2013-073R,EOA320-2013-073R,O,,,0,41400,,
1,B-6325,A320,2,EOA320-2013-082R,EOA320-2013-082R,O,,,0,49200,,
2,B-6325,A320,2,EOA320-2014-004R,EOA320-2014-004R,O,,,0,28700,,
3,B-6325,A320,2,EOA320-2015-154R,EOA320-2015-154R,O,,CS00960694,0,0,,2019-06-22
4,B-6325,A320,2,EOA320-2016-033R,EOA320-2016-033R02,O,,CS00960694,0,0,,2019-06-30
5,B-6325,A320,2,EOA320-2017-017R,EOA320-2017-017R,O,,CS00960694,0,0,,2019-06-30
6,B-6325,A320,2,EOA320-2017-023R,EOA320-2017-023R,O,,CS00960694,0,0,,2019-11-14
7,B-6325,A320,2,EOA320-2017-026R,EOA320-2017-026R,O,,CS00960694,0,0,,2021-12-31
8,B-6325,A320,2,EOA320-2017-046R,EOA320-2017-046R1,O,,CS00960694,0,0,,2019-11-15
9,B-6325,A320,2,EOA320-2017-070R,EOA320-2017-070R,O,,,0,0,,2021-08-30
